Code for developing and test the TimeSeriesDataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.utils

from janelia_core.ml.datasets import TimeSeriesDataset
from janelia_core.ml.datasets import TimeSeriesBatch
from janelia_core.ml.datasets import cat_time_series_batches

Create some TimeSeriesBatch objects and test the concatenation function

In [3]:
b1_data_g1 = torch.arange(10).view(10, 1)
b1_data_g1 = torch.cat([b1_data_g1, b1_data_g1], dim=1)
b1_data_g2 = torch.arange(10).view(10, 1).float() + .5

b1_i_x = torch.arange(9)
b1_i_y = b1_i_x + 1

b1_i_orig = torch.arange(10)

b1 = TimeSeriesBatch(data=[b1_data_g1, b1_data_g2], i_x=b1_i_x, 
                    i_y=b1_i_y, i_orig=b1_i_orig)


In [4]:
b2_data_g1 = torch.arange(5, 13).view(8, 1)
b2_data_g1 = torch.cat([b2_data_g1, b2_data_g1], dim=1)
b2_data_g2 = torch.arange(5, 13).view(8, 1).float() + .5

b2_i_x = torch.arange(7)
b2_i_y = b2_i_x + 1

b2_i_orig = torch.arange(5, 13)

b2 = TimeSeriesBatch(data=[b2_data_g1, b2_data_g2], i_x=b2_i_x, 
                    i_y=b2_i_y, i_orig=b2_i_orig)

In [5]:
c = cat_time_series_batches([b1, b2])

In [6]:
c.data[1][c.i_y]

tensor([[ 1.5000],
        [ 2.5000],
        [ 3.5000],
        [ 4.5000],
        [ 5.5000],
        [ 6.5000],
        [ 7.5000],
        [ 8.5000],
        [ 9.5000],
        [ 6.5000],
        [ 7.5000],
        [ 8.5000],
        [ 9.5000],
        [10.5000],
        [11.5000],
        [12.5000]])

In [7]:
b1.data[1][b1.i_y]

tensor([[1.5000],
        [2.5000],
        [3.5000],
        [4.5000],
        [5.5000],
        [6.5000],
        [7.5000],
        [8.5000],
        [9.5000]])

In [8]:
b2.data[1][b2.i_y]

tensor([[ 6.5000],
        [ 7.5000],
        [ 8.5000],
        [ 9.5000],
        [10.5000],
        [11.5000],
        [12.5000]])

Create a dataset and sample from it

In [9]:
ds = TimeSeriesDataset(ts_data=[b1_data_g1, b1_data_g2])

In [10]:
loader = torch.utils.data.DataLoader(dataset=ds, batch_size=2, 
                                     collate_fn=cat_time_series_batches,
                                     shuffle=True)

In [11]:
for b_i, b in enumerate(loader):
    print('******** Batch ' + str(b_i) + ' ********')
    print(b.data[0][b.i_x])

******** Batch 0 ********
tensor([[5, 5],
        [8, 8]])
******** Batch 1 ********
tensor([[1, 1],
        [3, 3]])
******** Batch 2 ********
tensor([[4, 4],
        [0, 0]])
******** Batch 3 ********
tensor([[6, 6],
        [2, 2]])
******** Batch 4 ********
tensor([[7, 7]])
